In [4]:
import pandas as pd

df_ac = pd.read_csv('../Data/Account.csv')
unique_account = set(df_ac['crm_Account_Account'])

df_con = pd.read_csv('../Data/Contact.csv')
unique_contact = set(df_con['crm_Contact_Contactpersoon'])

df_all = pd.read_csv('../Data/Afspraak alle.csv')
unique_afspraak = set(df_all['crm_Afspraak_ALLE_Afspraak'])

df_cam = pd.read_csv('../Data/Campagne.csv')
unique_campaign = set(df_cam['crm_Campagne_Campagne'])

df_mail = pd.read_csv("../Data/CDI mailing.csv")
unique_mail = set(df_mail["crm_CDI_Mailing_Mailing"])

df_per = pd.read_csv('../Data/Persoon.csv')
unique_persoon = set(df_per["crm_Persoon_persoon"])

df_func = pd.read_csv("../Data/Functie.csv")
unique_functie = set(df_func["crm_Functie_Functie"])

df_geb = pd.read_csv("../Data/Gebruikers.csv")
unique_gebruiker = set(df_geb['crm_Gebruikers_CRM_User_ID'])

df_insc = pd.read_csv("../Data/Inschrijving.csv")
unique_inschrijving = set(df_insc["crm_Inschrijving_Inschrijving"])

df_ses = pd.read_csv("../Data/Sessie.csv")
unique_sessie = set(df_ses["crm_Sessie_Sessie"])


In [6]:
df_fd = pd.read_csv("../Data/Account financiële data.csv")
df_fd = df_fd[df_fd['crm_FinancieleData_OndernemingID'].isin(unique_account)]
df_fd = df_fd.drop_duplicates(subset=['crm_FinancieleData_OndernemingID', 'crm_FinancieleData_Boekjaar'], keep='first')

df_fd.to_csv("../Data/Account financiële data.csv", index=False)

In [12]:
df_insc = df_insc[df_insc['crm_Inschrijving_Contactfiche'].isin(unique_contact)]
df_insc = df_insc[df_insc["crm_Inschrijving_Bron"].isin(unique_campaign)]
df_insc.drop_duplicates(subset='crm_Inschrijving_Inschrijving', inplace=True, keep='first')

df_insc.to_csv("../Data/Inschrijving.csv", index=False)

In [15]:
df_pa = pd.read_csv("../Data/cdi pageviews.csv", delimiter=";")

df_pa.drop('crm CDI_PageView[Anonymous Visitor]', inplace=True, axis=1)
df_pa.drop('crm CDI_PageView[Url]', inplace=True, axis=1)
df_pa.drop_duplicates(keep='first', inplace=True)

df_pa = df_pa[df_pa['crm CDI_PageView[Contact]'].isin(unique_contact)]
df_pa = df_pa[df_pa['crm CDI_PageView[Campaign]'].isin(unique_campaign)]

df_pa.to_csv("../Data/cdi pageviews.csv", index=False)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 10080, saw 2


In [16]:
df_vi = pd.read_csv("../Data/CDI visits.csv")

df_vi.drop("crm_CDI_Visit_Campagne_Code", axis=1, inplace=True)
df_vi.drop_duplicates(subset=["crm_CDI_Visit_Visit","crm_CDI_Visit_Contact" ], keep="first", inplace=True)

df_vi = df_vi[df_vi["crm_CDI_Visit_Contact"].isin(unique_contact)]
df_vi = df_vi[df_vi["crm_CDI_Visit_Email_Send"].isin(unique_mail)]
df_vi = df_vi[df_vi["crm_CDI_Visit_Campaign"].isin(unique_campaign)]

df_vi.to_csv("../Data/CDI visits.csv", index=False)

/tmp/ipykernel_20528/1316398967.py:1: DtypeWarning: Columns (4,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vi = pd.read_csv("../Data/CDI visits.csv")


In [17]:
df_con = df_con[df_con['crm_Contact_Account'].isin(unique_account)]
df_con = df_con[df_con['crm_Contact_Persoon_ID'].isin(unique_persoon)]
df_con.drop_duplicates('crm_Contact_Contactpersoon', keep='first', inplace=True)

df_con.to_csv('../Data/Contact.csv', index=False)

In [18]:
df_gc = pd.read_csv('../Data/Afspraak_account_gelinkt_cleaned.csv')

df_gc = df_gc[df_gc['crm_Afspraak_ACCOUNT_GELINKT_Afspraak'].isin(unique_afspraak)]
df_gc.drop_duplicates('crm_Afspraak_ACCOUNT_GELINKT_Afspraak',keep='first', inplace=True)

df_gc.to_csv('../Data/Afspraak_account_gelinkt_cleaned.csv', index=False)

In [19]:
df_cc = pd.read_csv("../Data/Afspraak betreft contact_cleaned.csv")

df_cc = df_cc[df_cc['crm_Afspraak_BETREFT_CONTACTFICHE_Betreft_id'].isin(unique_contact)]
df_cc = df_cc[df_cc['crm_Afspraak_BETREFT_CONTACTFICHE_Afspraak'].isin(unique_afspraak)]
df_cc.drop_duplicates(['crm_Afspraak_BETREFT_CONTACTFICHE_Betreft_id', 'crm_Afspraak_BETREFT_CONTACTFICHE_Afspraak'],keep='first' ,inplace=True )

df_cc.to_csv("../Data/Afspraak betreft contact_cleaned.csv", index=False)

In [21]:
df_vc = pd.read_csv("../Data/Activiteit vereist contact.csv")

df_vc = df_vc[df_vc["crm_ActiviteitVereistContact_ActivityId"].isin(unique_afspraak)]
df_vc = df_vc[df_vc["crm_ActiviteitVereistContact_ReqAttendee"].isin(unique_contact)]
df_vc.drop_duplicates(['crm_ActiviteitVereistContact_ReqAttendee', 'crm_ActiviteitVereistContact_ActivityId'], keep="first", inplace=True)

df_vc.to_csv("../Data/Activiteit vereist contact.csv", index=False)

In [22]:
df_acc = pd.read_csv("../Data/Afspraak betreft account_cleaned.csv")

df_acc = df_acc[df_acc['crm_Afspraak_BETREFT_ACCOUNT_Betreft_id'].isin(unique_account)]
df_acc = df_acc[df_acc['crm_Afspraak_BETREFT_ACCOUNT_Afspraak'].isin(unique_afspraak)]
df_acc.drop_duplicates(keep='first', inplace=True)

df_acc.to_csv("../Data/Afspraak betreft account_cleaned.csv", index=False)

In [24]:
df_cf = pd.read_csv("../Data/Contact functie.csv")

df_cf = df_cf[df_cf['crm_ContactFunctie_Functie'].isin(unique_functie)]
df_cf = df_cf[df_cf['crm_ContactFunctie_Contactpersoon'].isin(unique_contact)]
df_cf.drop_duplicates(keep='first', inplace=True)

df_cf.to_csv("../Data/Contact functie.csv", index=False)

In [26]:
df_sent = pd.read_csv("../Data/CDI sent email clicks.csv")

df_sent = df_sent[df_sent['crm_CDI_SentEmail_kliks_Contact'].isin(unique_contact)]
df_sent = df_sent[df_sent['crm_CDI_SentEmail_kliks_E_mail_versturen'].isin(unique_mail)]
df_sent.drop_duplicates(keep='first', inplace=True)

df_sent.to_csv("../Data/CDI sent email clicks.csv", index=False)

In [28]:
df_info = pd.read_csv("../Data/Info en klachten.csv")

df_info = df_info[df_info["crm_Info_en_Klachten_Eigenaar"].isin(unique_gebruiker)]
df_info = df_info[df_info["crm_Info_en_Klachten_Account"].isin(unique_account)]
df_info.drop_duplicates(keep="first", inplace=True)

df_info.to_csv("../Data/Info en klachten.csv", index=False)

In [29]:
df_lid = pd.read_csv("../Data/Lidmaatschap.csv")

df_lid = df_lid[df_lid["crm_Lidmaatschap_Onderneming"].isin(unique_account)]
df_lid.drop_duplicates(keep="first", inplace=True)

df_lid.to_csv("../Data/Lidmaatschap.csv", index=False)

In [30]:
df_ses = df_ses[df_ses["crm_Sessie_Campagne"].isin(unique_campaign)]
df_ses.drop_duplicates(keep='first', inplace=True)

df_ses.to_csv("../Data/Sessie.csv", index=False)

In [32]:
df_ses_in = pd.read_csv("../Data/Sessie inschrijving.csv")

df_ses_in = df_ses_in[df_ses_in["crm_SessieInschrijving_Sessie"].isin(unique_sessie)]
df_ses_in = df_ses_in[df_ses_in["crm_SessieInschrijving_Inschrijving"].isin(unique_inschrijving)]
df_ses_in.drop_duplicates(keep="first", inplace=True)

df_ses_in.to_csv("../Data/Sessie inschrijving.csv", index=False)